# Reading the Data:

In [ ]:
# Importing Pandas and Reading the Data:
import pandas as pd
data = pd.read_csv("../input/building-permit-applications-data/Building_Permits.csv")
data = data.iloc[0:10000,:]
data.head(5)

# Preprocessing

In [ ]:
# Lets take a look at the null values in the data:
null = pd.DataFrame(pd.isnull(data).sum())
null

## Removing Unnecessary Columns:

I think the first step to starting to work on something which as a lot of columns is by removing the columns which are unnecessary given that they provide no value to the data but rather make it messy:

In [ ]:
# Instead of coursing throught the 43 columns, lets make a list of the features which have more than 10K null values:
droplist = []
for i in null.index:
    if int(null.loc[i].values) > 2000:
        droplist.append(i)

In [ ]:
# Here is how it looks:
droplist.append('Description')
droplist

In [ ]:
# Finally dropping the useless columns:
newdata = data.drop(droplist, axis = 1)

## Filling in Missing Values:

In [ ]:
# Defining the Cateogircal Columns:
c = (newdata.dtypes == 'object')
catcol = list(c[c].index)
catcol

In [ ]:
# Defining the Numerical Columns:
n = (newdata.dtypes != 'object')
numcol = list(n[n].index)
numcol

In [ ]:
# Now we must deal with the features who have null values but less than 10K in count:
from sklearn.impute import SimpleImputer
CatSimp = SimpleImputer(strategy = 'most_frequent')

CatSimp.fit(newdata[catcol])
newdata[catcol] = CatSimp.transform(newdata[catcol])

In [ ]:
# Its time to fill in the missing values in the Numerical Columns:
NumSimp = SimpleImputer(strategy = 'median')

NumSimp.fit(newdata[numcol])
newdata[numcol] = NumSimp.transform(newdata[numcol])

Before converting my categorical columns to numerical to process it for insights, I'll copy the dataset first so that I don't make changes to the newdata dataset.

## Dealing with Special Columns:

In [ ]:
# Finding longitudes and latitudes:
latlong = newdata['Location'].str.strip('()').str.split(', ', expand = True)
newdata['Latitude'] = latlong[[0]]
newdata['Longitude'] = latlong[[1]]

newdata.drop(['Location'], axis = 1, inplace =True)

In [ ]:
# Defining a function which produces day, month and year columns from a single date column:

from datetime import date

def getdate(x, c):
    d = []
    m = []
    y = []
    x[c] = pd.to_datetime(x[c])
    for n in range(0,10000):
        d.append(x[c].iloc[n].day)
        m.append(x[c].iloc[n].month)
        y.append(x[c].iloc[n].year)
        
        
    x[c + " " + "Day"] = d
    x[c + " " + "Month"] = m
    x[c + " " + "Year"] = y
    x.drop([c], axis = 1, inplace = True)
    return 

In [ ]:
# Defining dates as the collection all the features with dates:
dates = ['Permit Creation Date', 'First Construction Document Date', 'Filed Date', 'Current Status Date', 'Issued Date']

# Lets produce some day, month and year columns:
for d in dates:
    getdate(newdata, d)

In [ ]:
# Since the Values of Longitude and Latitudes were in the "object" dtype, lets convert it to float:
newdata['Longitude'] = newdata['Longitude'].astype('float64')
newdata['Latitude'] = newdata['Latitude'].astype('float64')

In [ ]:
# I dont't want to mess up the newdata dataset since converting its categorical values into numerical values will affect data visualization,
# lets copy it to another dataset:
encdata = pd.DataFrame(newdata, columns = newdata.columns)
PermitNumber = encdata['Permit Number']
encdata.drop(['Permit Number'], axis = 1, inplace = True)

## Converting Categorical Values:

In [ ]:
# Defining a function that converts a dataset's categorical values to Numerical Values
def convertcat(data, target):

# Defining the Categorical Columns:
    c = (data.dtypes == 'object')
    datacatcol = list(c[c].index)


# Importing the bad boy needed to convert Categorical Values to Numerical Values:
    from category_encoders import CatBoostEncoder
    cbe = CatBoostEncoder()

# CatBoostEncoder demands a target value becuase it changes the cate
    for i in datacatcol:
        cbe.fit(data[i], data[target])
        data[i] = cbe.transform(data[i], data[target])

    return

In [ ]:
# Convertin encdat:
convertcat(encdata, 'Estimated Cost')

# Data Visualization and Analysis:

## HeatMap
Here's a correlation heatmap which will help us get insights on the data:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize  = (17, 20))
sns.set_context("paper", font_scale = 1)
sns.heatmap(encdata.corr())

## Scatter Maps:

In [ ]:
# Visualizing the permits on a map with their Current Status:

import plotly.express as px

fig = px.scatter_mapbox(newdata, lat = "Latitude", lon="Longitude",
                        zoom = 10, height = 300, 
                        color = "Current Status")
fig.update_layout(mapbox_style = "carto-positron")
fig.update_layout(margin = {"r":0, "t":0, "l":0, "b":0})
fig.show()

In [ ]:
# Visualizing this again but on a map to have a better idea:

import plotly.express as px

fig = px.scatter_mapbox(newdata, lat = "Latitude", lon="Longitude",
                        zoom = 10, height = 300, 
                        color = "Plansets")
fig.update_layout(mapbox_style = "carto-positron")
fig.update_layout(margin = {"r":0, "t":0, "l":0, "b":0})
fig.show()

What I can conclude from above is that the north west regions have more plansets, clustered around the San Francisco-Oakland bridge.

## Countplots:

In [ ]:
# Plotting a countplot for the Current Status' of permits:

plt.figure(figsize = (20, 6))
sns.set_context("poster", font_scale = 0.9)
sns.countplot(newdata['Current Status'])

In [ ]:
# Plotting the Countplot for Filed Date Month:

plt.figure(figsize=(12,6))
sns.set_context("poster")
sns.countplot(newdata['Filed Date Month'])

In [ ]:
# Plotting the Countplot for Issued Date Month:

plt.figure(figsize=(12,6))
sns.set_context("poster")
sns.countplot(newdata['Issued Date Month'])

It is plausible to conclude by just looking at the two countplots above that most **filings** and **issues** occur during the **first four months** of a year.

In [ ]:
# Countplot for the days on which permits are issued:

plt.figure(figsize=(20,5))
sns.countplot(newdata['Issued Date Day'])

The Fourth of every months seems to be the favourite date of the authorities for issuing permits.

I would like to thank you for going through this notebook :D
Check out my other notebooks if you feel like!
